In [ ]:
import importlib
import functions.core_functions as core_functions
import dask
import dask.dataframe as dd
from dask.dataframe.utils import assert_eq
import dask_cudf as dc
import pandas as pd
import numpy as np
import pandas_gbq

dask.config.set({"dataframe.backend": "cudf"})



importlib.reload(core_functions)


resp = {}
resp = core_functions.initialize_clients()

config = resp.get('config')
bigquery_client = resp.get('clients').get('bigquery_client')
storage_client = resp.get('clients').get('storage_client')
sf_client = resp.get('clients').get('sf_client')
veil_billing = resp.get('config').get('veil_billing')
veil_vars = resp.get('config').get('veil_billing').get('vars')
# print(veil_billing)



In [ ]:


target_table = 'encodings'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
encodings_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'encoders'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
encoders_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'encoder_groups'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
encoder_groups_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)


target_table = 'formats'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
formats_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'customers'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
customers_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'profiles'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
profiles_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'aeismaps'
fetch_sql = f"""
SELECT * from {veil_billing.get('avs_project_id')}.{veil_billing.get('avs_dataset_id')}.{target_table}"""
aeismaps_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)


target_table = 'sfdc_bvs_customer__c_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_bvs_customer_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'sfdc_bvs_format__c_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_bvs_format_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'sfdc_account_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_account_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'sfdc_advertiser__c_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_advertiser_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'sfdc_rate_card__c_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_rate_card_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)

target_table = 'sfdc_rate_card__c_obj'
fetch_sql = f"""
SELECT * from {veil_billing.get('billing_project_id')}.{veil_billing.get('billing_dataset_id')}.{target_table}"""
sfdc_rate_card_df = core_functions.fetch_gbq_data(fetch_sql, bigquery_client)



In [ ]:
attributes_list = ['product_code', 'product_name', 'donovan_agency_product_code', 'description', 'isci', 'project_name', 'advertiser', 'client_code',
                   'cable_estimate', 'spot_estimate', 'campaign', 'audience', 'audience2', 'category', 'comercial_id', 'contour_id', 'creative_offer',
                   'donovan_agency_advertiser_code', 'donovan_agency_estimate_code', 'eid', 'group', 'hd_sd', 'id', 'length', 'lob', 'media_type',
                   'message', 'misc', 'module_code', 'offer', 'offer_2', 'phone_number', 'quality', 'revision', 'show_name', 'slug', 'sport_id',
                   'sport_show_sub_category', 'spot_name', 'tag', 'text', 'title', 'veil_id', 'version_name', 'year']
for attr in attributes_list:
    encodings_df[attr] = encodings_df['attributes'].apply(lambda x: x.get(attr))
# encodings_df['product_code'] = encodings_df['attributes'].apply(lambda x: x.get('product_code'))

# Define conditions
# Ensure 'description' column does not contain None values
encodings_df['description'] = encodings_df['description'].fillna('')

# Define conditions
conditions = [
    encodings_df['product_code'].notnull(),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].notnull(),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].isnull() & encodings_df['donovan_agency_product_code'].notnull(),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].isnull() & encodings_df['donovan_agency_product_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & ~encodings_df['description'].str.startswith(('TV', 'RA')),
    encodings_df['product_code'].isnull() & encodings_df['product_name'].isnull() & encodings_df['donovan_agency_product_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & encodings_df['description'].str.startswith(('TV', 'RA'))
]

# Define corresponding values
choices = [
    encodings_df['product_code'],
    encodings_df['product_name'],
    encodings_df['donovan_agency_product_code'],
    encodings_df['description'].str[26:30].str.strip(),
    encodings_df['description'].str[6:10].str.strip()
]

# Apply conditions and choices to create the new column
encodings_df['product_code'] = np.select(conditions, choices, default=None)



# Define conditions
conditions = [
    encodings_df['isci'].notnull(),
    encodings_df['isci'].isnull() & encodings_df['project_name'].notnull(),
    encodings_df['isci'].isnull() & encodings_df['project_name'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & ~encodings_df['description'].str.startswith(('TV', 'RA')),
    encodings_df['isci'].isnull() & encodings_df['project_name'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & encodings_df['description'].str.startswith(('TV', 'RA'))
]

# Define corresponding values
choices = [
    encodings_df['isci'],
    encodings_df['project_name'],
    encodings_df['description'].str[8:18].str.strip(),
    encodings_df['description'].str[18:38].str.strip()
]

# Apply conditions and choices to create the new column
encodings_df['isci'] = np.select(conditions, choices, default=None)


# Define conditions
conditions = [
    encodings_df['advertiser'].notnull(),
    encodings_df['advertiser'].isnull() & encodings_df['client_code'].notnull(),
    encodings_df['advertiser'].isnull() & encodings_df['client_code'].isnull() & encodings_df['donovan_agency_advertiser_code'].notnull(),
    encodings_df['advertiser'].isnull() & encodings_df['project_name'].isnull() & encodings_df['donovan_agency_advertiser_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & ~encodings_df['description'].str.startswith(('TV', 'RA')),
    encodings_df['advertiser'].isnull() & encodings_df['project_name'].isnull() & encodings_df['donovan_agency_advertiser_code'].isnull() & encodings_df['description'].notnull() & encodings_df['description'].str.len() > 10 & encodings_df['description'].str.startswith(('TV', 'RA'))
]

# Define corresponding values
choices = [
    encodings_df['advertiser'],
    encodings_df['client_code'],
    encodings_df['donovan_agency_advertiser_code'],
    encodings_df['description'].str[22:26].str.strip(),
    encodings_df['description'].str[2:6].str.strip()
]

# Apply conditions and choices to create the new column
encodings_df['advertiser'] = np.select(conditions, choices, default=None)
encodings_df

In [ ]:
# def rename_columns(df, prefix):
#     for col in df.columns:
#         if col.startswith(prefix):
#             continue
#         else:
#             df[f"{prefix}_{col}"] = df[col]
#             df.drop(columns=[col], inplace=True)
#     return df

core_functions.reset_dataframe()
encodings_df.head()


In [ ]:
core_functions.rename_columns(encoder_groups_df, 'encoder_group_')
core_functions.rename_columns(encoders_df, 'encoder_')
core_functions.rename_columns(formats_df, 'format_')
core_functions.rename_columns(customers_df, 'customer_')
core_functions.rename_columns(profiles_df, 'profile_')
core_functions.rename_columns(aeismaps_df, 'aeis_')

encoders_groups_df = encoders_df.merge(encoder_groups_df, left_on='encoder_group_id', right_on='encoder_group_id', how='left')
encoders_groups_df


encodings_encoders_df = encodings_df.merge(encoders_groups_df, left_on='encoder_id', right_on='encoder_id', how='left', suffixes=('', '_dupe'))
encodings_encoders_df

aeismaps_df.sort_values(by=['aeis__encoding_id', 'aeis_id', ], inplace=True)
aeismaps_df = aeismaps_df.loc[(aeismaps_df['aeis_id'].fillna(0.0).astype(int) > 0) & (aeismaps_df['aeis__encoding_id'].fillna(0.0).astype(int) > 0)]
aeismaps_df.drop_duplicates(subset=('aeis__encoding_id' ), inplace=True)
aeismaps_df

encodings_aeis_df = encodings_encoders_df.merge(aeismaps_df, left_on='encoding_id', right_on='aeis__encoding_id', how='left')
encodings_aeis_df

formats_customers_df = formats_df.merge(customers_df, left_on='format__customer_id', right_on='customer_id', how='left')

formats_customers_profiles_df = formats_customers_df.merge(profiles_df, left_on='format__profile_id', right_on='profile_id', how='left')
formats_customers_profiles_df

encodings_bvs_df = encodings_aeis_df.merge(formats_customers_profiles_df, left_on='format_id', right_on='format_id', how='left')
encodings_bvs_df

encodings_bvs_df['ad_prod_campaign'] = None
encodings_bvs_df['advertiser'] = encodings_bvs_df['advertiser'].fillna('')
encodings_bvs_df['product_code'] = encodings_bvs_df['product_code'].fillna('')
encodings_bvs_df['campaign'] = encodings_bvs_df['campaign'].fillna('')

# Update ad_prod_campaign
encodings_bvs_df['ad_prod_campaign'] = encodings_bvs_df.apply(
    lambda row: f"{row['advertiser'].strip()}-{row['product_code'].strip()}-{row['campaign'].strip()}".replace(' ', '_') if pd.isnull(row['ad_prod_campaign']) else row['ad_prod_campaign'],
    axis=1
)


In [ ]:
# Sample DataFrames
# encodings_bvs_df = pd.DataFrame(...)
# sfdc_advertiser_df = pd.DataFrame(...)




# # Define a function to perform the updates
# def update_encodings_bvs_df(encodings_bvs_df, sfdc_advertiser_df):
#     # First update
#     mask = encodings_bvs_df['ad_prod_campaign'].isnull()
#     encodings_bvs_df.loc[mask, 'ad_prod_campaign'] = encodings_bvs_df[mask].apply(
#         lambda row: f"{row['advertiser'].strip()}-{row['product_code'].strip()}-{row['campaign'].strip()}".replace(' ', '_'),
#         axis=1
#     )

#     # Define a function to perform the updates based on conditions
#     def update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, left_on, right_on, match_type):
#         merged_df = encodings_bvs_df.merge(sfdc_advertiser_df, how='left', left_on=left_on, right_on=right_on)
#         mask = (merged_df['sfdc_advertiser_id'].isnull()) & (merged_df['Encoding_Format_ID__c'].notnull()) & (merged_df['format_id'] == merged_df['Encoding_Format_ID__c']) & (merged_df['Match_Type__c'] == match_type)
#         encodings_bvs_df.loc[mask, 'sfdc_advertiser_id'] = merged_df.loc[mask, 'Id']
#         encodings_bvs_df.loc[mask, 'sfdc_advertiser_name'] = merged_df.loc[mask, 'Name']
#         encodings_bvs_df.loc[mask, 'sfdc_advertiser_match_type'] = merged_df.loc[mask, 'Match_Type__c']
#         encodings_bvs_df.loc[mask, 'sfdc_rate_card_id'] = merged_df.loc[mask, 'Related_Rate_Card__c']
#         encodings_bvs_df.loc[mask, 'advertiser_updated'] = pd.Timestamp.now()

#     # Apply updates based on different conditions
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'product_code', 'Enc_Product_Code__c', 'encoding_product_code_multiple')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'product_code', 'Enc_Product_Code__c', 'encoding_product_code')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'product_code', 'Enc_Product_Code__c', 'encoding_product_code_ignore_format')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'advertiser', 'Enc_Advertiser__c', 'encoding_advertiser')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'advertiser', 'Enc_Advertiser__c', 'encoding_advertiser_ignore_format')
#     update_with_conditions(encodings_bvs_df, sfdc_advertiser_df, 'format_id', 'Encoding_Format_ID__c', 'encoding_format')

# # Call the function to update the DataFrame
# update_encodings_bvs_df(encodings_bvs_df, sfdc_advertiser_df)

# print(encodings_bvs_df)

In [ ]:
mask = sfdc_advertiser_df['Match_Type__c'] == 'encoding_format'
print(len(sfdc_advertiser_df.loc[mask]))
encoding_format_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()

mask = sfdc_advertiser_df['Match_Type__c'] == 'encoder_group'
print(len(sfdc_advertiser_df.loc[mask]))
encoder_group_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()

mask = sfdc_advertiser_df['Match_Type__c'] == 'encoder_group'
print(len(sfdc_advertiser_df.loc[mask]))
encoder_group_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()

mask = sfdc_advertiser_df['Match_Type__c'] == 'encoder_group'
print(len(sfdc_advertiser_df.loc[mask]))
encoder_group_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()

mask = sfdc_advertiser_df['Match_Type__c'] == 'encoder_group'
print(len(sfdc_advertiser_df.loc[mask]))
encoder_group_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()

mask = sfdc_advertiser_df['Match_Type__c'] == 'encoder_group'
print(len(sfdc_advertiser_df.loc[mask]))
encoder_group_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()

mask = sfdc_advertiser_df['Match_Type__c'] == 'encoder_group'
print(len(sfdc_advertiser_df.loc[mask]))
encoder_group_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()

mask = sfdc_advertiser_df['Match_Type__c'] == 'encoder_group'
print(len(sfdc_advertiser_df.loc[mask]))
encoder_group_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()

mask = sfdc_advertiser_df['Match_Type__c'] == 'encoder_group'
print(len(sfdc_advertiser_df.loc[mask]))
encoder_group_sfdc_advertiser_df = sfdc_advertiser_df.loc[mask].copy()




# array(['encoding_format', 'encoder_group', 'Clone',
    #    'encoding_product_code_multiple', None, 'encoding_advertiser',
    #    'encoding_product_code', 'encoding_advertiser_ignore_format'],
    #   dtype=object)
# 14327 total
# encoding_format: 14174
# encoder_group: 32
# Clone: 2
# encoding_product_code_multiple: 17
# encoding_advertiser: 7
# encoding_product_code: 37
# encoding_advertiser_ignore_format: 45
#  null: 13
print(14327 - 14174 - 32 - 2 - 17 - 7 - 37 - 45 - 13)
sfdc_advertiser_df.loc[mask]

In [ ]:
# profiles_df.loc[profiles_df['profile__default_asset_code'] != 'isci']

# expanded_df = profiles_df['profile__attributes'].apply(pd.Series)

# # Combine the original DataFrame with the expanded DataFrame
# result_df = pd.concat([profiles_df, expanded_df], axis=1)
# core_functions.show_more_dataframe()
# result_df
        # CASE WHEN (e.attributes.product_code is NOT NULL) THEN e.attributes.product_code
        # WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NOT NULL) THEN e.attributes.product_name
        # WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NULL AND e.attributes.donovan_agency_product_code is NOT NULL) THEN e.attributes.donovan_agency_product_code
        # WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NULL AND e.attributes.donovan_agency_product_code is NULL and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description not like 'TV%' or e.attributes.description not like 'RA%')) THEN trim(substring(e.attributes.description, 27,4))
        # WHEN (e.attributes.product_code is NULL AND e.attributes.product_name is NULL AND e.attributes.donovan_agency_product_code is NULL and e.attributes.description is not null and length(e.attributes.description) > 10 and (e.attributes.description  like 'TV%' or e.attributes.description  like 'RA%')) THEN trim(substring(e.attributes.description, 7,4))
        # ELSE NULL END AS product_code,
